In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install seaborn

More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)

Using TensorFlow backend.


In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)

In [0]:
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
embed_size

512

In [0]:
import pandas as pd
Output_df = pd.read_excel("./drive/My Drive/cleansed_organic/GPT2_statements_output.xlsx")
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(Output_df, test_size=0.2)

In [0]:

df_train.Label = df_train.Label.astype('category')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [0]:
df_train.head()

,Output,Label
1254,The Organic food system suffers from the agroi...,n
744,"Organic products are expensive, probably bett...",n
990,Organic label criteria are appropriate for a ...,n
661,Organic products contain no or less chemical ...,n
1839,Organic food yields cannot feed the world; it...,0


In [0]:
category_counts = len(df_train.Label.cat.categories)
category_counts

3

In [0]:
print(df_train.groupby('Label').size())

Label
0    534
n    525
p    525
dtype: int64


## Wrap embed module in a Lambda layer
Explicitly cast the input as a string

In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [0]:
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(category_counts, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 132,099
Trainable params: 132,099
Non-trainable params: 0
_________________________________________________________________


In [0]:
train_text = df_train['Output'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(df_train.Label), dtype = np.int8)

In [0]:
train_text.shape

(1584, 1)

In [0]:
train_label.shape

(1584, 3)

In [0]:
train_label[:3]

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=int8)

In [0]:
# df_test = pd.read_pickle("./drive/My Drive/cleansed_organic/cleansed_fine_tune_organic_full_validation.pkl")
df_test.Label = df_test.Label.astype('category')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [0]:
test_text = df_test['Output'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = np.asarray(pd.get_dummies(df_test.Label), dtype = np.int8)

In [0]:
test_text.shape

(396, 1)

In [0]:
test_label.shape

(396, 3)

## Train Keras model and save weights
This only train and save our Keras layers not the embed module' weights.

In [0]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=16,
            batch_size=128)
  model.save_weights('./drive/My Drive/cleansed_organic/model5.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1584 samples, validate on 396 samples
Epoch 1/16
1584/1584 [==============================] - 11s 7ms/step - loss: 1.0702 - acc: 0.4874 - val_loss: 1.0330 - val_acc: 0.5328
Epoch 2/16
1584/1584 [==============================] - 8s 5ms/step - loss: 1.0021 - acc: 0.6143 - val_loss: 0.9647 - val_acc: 0.6768
Epoch 3/16
1584/1584 [==============================] - 8s 5ms/step - loss: 0.9331 - acc: 0.6471 - val_loss: 0.9009 - val_acc: 0.6717
Epoch 4/16
1584/1584 [==============================] - 8s 5ms/step - loss: 0.8744 - acc: 0.6717 - val_loss: 0.8367 - val_acc: 0.7273
Epoch 5/16
1584/1584 [==============================] - 8s 5ms/step - loss: 0.8151 - acc: 0.7020 - val_loss: 0.7862 - val_acc: 0.7298
Epoch 6/16
1584/1584 [==============================] - 8s 5ms/step - loss: 0.7675 - acc: 0.7077 - val_loss: 0.7433 - val_acc: 0.7374
Epoch 7/16
1584/1584 [==============================] - 8s 5ms/step - loss: 0.7298 - acc: 0.7260 - val_loss: 0.7090 - val_acc: 0.7475
Epoch 8/16
158

In [0]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=13,
            batch_size=128)
  model.save_weights('./drive/My Drive/cleansed_organic/model6.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1584 samples, validate on 396 samples
Epoch 1/13
1584/1584 [==============================] - 11s 7ms/step - loss: 1.0702 - acc: 0.4874 - val_loss: 1.0330 - val_acc: 0.5328
Epoch 2/13
1584/1584 [==============================] - 7s 5ms/step - loss: 1.0021 - acc: 0.6143 - val_loss: 0.9647 - val_acc: 0.6768
Epoch 3/13
1584/1584 [==============================] - 7s 5ms/step - loss: 0.9331 - acc: 0.6471 - val_loss: 0.9009 - val_acc: 0.6717
Epoch 4/13
1584/1584 [==============================] - 7s 5ms/step - loss: 0.8744 - acc: 0.6717 - val_loss: 0.8367 - val_acc: 0.7273
Epoch 5/13
1584/1584 [==============================] - 7s 5ms/step - loss: 0.8151 - acc: 0.7020 - val_loss: 0.7862 - val_acc: 0.7298
Epoch 6/13
1584/1584 [==============================] - 7s 5ms/step - loss: 0.7675 - acc: 0.7077 - val_loss: 0.7433 - val_acc: 0.7374
Epoch 7/13
1584/1584 [==============================] - 7s 5ms/step - loss: 0.7298 - acc: 0.7260 - val_loss: 0.7090 - val_acc: 0.7475
Epoch 8/13
158

## Make predictions

In [0]:
new_text = []
belief_pd = pd.read_pickle("./drive/My Drive/cleansed_organic/cleansed_fine_tune_organic_products_test.pkl")
belief_pd['Output_prediction'] = ''
for index, row in belief_pd.iterrows():
  new_text.append(row['Sentence']) 


new_text = np.array(new_text, dtype=object)[:, np.newaxis]

print('new_text', new_text)

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./drive/My Drive/cleansed_organic/model6.h5')  
  predicts = model.predict(new_text, batch_size=32)
  

  
categories = df_train.Label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)

# print('predict_logits', predict_logits)

predict_labels = [categories[logit] for logit in predict_logits]

print('predict_labels', predict_labels)
for index, row in belief_pd.iterrows():
#   belief_pd.at[index,'Output_prediction'] = '0'
#   print(index)
  belief_pd.at[index,'Output_prediction'] = str(predict_labels[index])
  

new_text [['Muir Glen Organic uses BPA free cans and it is stated on the back of the can in small print "The lining of this can was produced without BPA."']
 ['Beware that just because a product is organic does not mean the can is BPA free.']
 ['We hear it time and again that organic food is better for health.']
 ['While shopping, we even look for the ‘organic’ label on the products.']
 ['But have we ever thought that why organic products are good?']
 ['It is proved by various studies that organic products have certain benefits but to what extent that isn’t always clear.']
 ['Some organic products are definitely more important than others and organic tea is no doubt a smarter choice than conventional tea.']
 ['Check out the reasons for buying and drinking organic tea are: Chemical free- The first reason to buy organic tea is that it is grown without the using any harmful chemical- fertilizer, pesticides, fungicides, or herbicides.']
 ['As proved by various researchers, these chemicals 

IndexError: ignored

In [0]:
belief_pd.to_excel('./drive/My Drive/cleansed_organic/GPT2_statements_prediction_output.xlsx', index = None, header=True, encoding='utf-8-sig')